In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir( os.path.join('..', 'notebook_format') )
from formats import load_style
load_style()

In [2]:
os.chdir(path)
import pandas as pd

# Pandas Category

Make pandas DataFrame smaller and faster using category type.

In [3]:
drinks = pd.read_csv('http://bit.ly/drinksbycountry')
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [4]:
# We'll first look at the memory usage of each column
drinks.memory_usage(deep = True)

Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                       12332
dtype: int64

In [5]:
# and the type of each column
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 9.1+ KB


Since the `country` and `continent` columns are strings, they are represented as object types in pandas. Now let's say, instead of storing strings, we want to store the `continent` column as integers to reduce the memory required to store them, e.g. like in the unique continent below, we'll store Asia as 0, Europe as 1 and so on. Then we'll simply store another lookup table, to make sure that we can convert the integer back to the original continent.

To apply this notion, we simply have to convert the column type to `category`.

In [6]:
drinks['continent'].unique()

array(['Asia', 'Europe', 'Africa', 'North America', 'South America',
       'Oceania'], dtype=object)

In [7]:
# convert and print the memory usage
drinks['continent'] = drinks['continent'].astype('category')
drinks.memory_usage(deep = True)

Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         584
dtype: int64

In [8]:
# list out the unique categories
drinks['continent'].cat.categories

Index(['Africa', 'Asia', 'Europe', 'North America', 'Oceania',
       'South America'],
      dtype='object')

We can see that by converting the `continent` column to integers we're being more space-efficient (it is actually faster if you do other operations on it, e.g. sorting, groupby) as we're storing the strings as integers. Let's apply this notion again to the `country` column.

In [9]:
drinks['country'] = drinks['country'].astype('category')
drinks.memory_usage(deep = True)

Index                              80
country                         12974
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                         584
dtype: int64

This time, the memory usage for the country column is now larger. The reason is that the country column's value is unique. Thus we're actually creating 193 (shown below) unqiue categories, and we also have to store a lookup table for that.

In [10]:
drinks['country'].cat.categories.shape

(193,)

In summary, if we're working with an object column of strings, convert it to `category` type to make it for efficient. But this must be based on the assumption that the column takes a limited number of unique values, like in this case, the continent column only has 6 unique values.

Another usage of `category` is to specify its order to perform sorting.

In [11]:
# toy dataset
df = pd.DataFrame({
    'ID': [100, 101, 102, 103], # id for a product
    'quality': ['good', 'very good', 'good', 'excellent'] # customer review for it
})
df

,ID,quality
0,100,good
1,101,very good
2,102,good
3,103,excellent


In [12]:
# if we do a sort on quality, it will be
# sorted alphabetically (default sorting for strings)
df.sort_values('quality')

,ID,quality
3,103,excellent
0,100,good
2,102,good
1,101,very good


In [13]:
# now we can specify our own ordering
df['quality'] = df['quality'].astype('category', 
                                     categories = ['good', 'very good', 'excellent'], 
                                     ordered = True)
df.sort_values('quality')

,ID,quality
0,100,good
2,102,good
1,101,very good
3,103,excellent


In [14]:
# we can even use boolean method on this new ordering
df.loc[df['quality'] > 'good']

,ID,quality
1,101,very good
3,103,excellent


## Reference

- [Youtube: How do I make my pandas DataFrame smaller and faster?](https://www.youtube.com/watch?v=wDYDYGyN_cw)